In [164]:
import pandas as pd
import numpy as np
import random
import math
from tqdm import tqdm

In [165]:
# data = pd.read_csv(open('data/room.csv'))
# data.replace('?', np.nan).fillna(method='pad'))
data = pd.read_csv('data/horse/train.csv')
feature_columns = [ col for col in data.columns if col not in ['id','outcome']]
data[feature_columns].nunique()
# encode labels column to numbers
# y = data['outcome']

surgery                         2
Age                             2
Hospital ID                   284
rectal temperature             41
pulse                          53
respiratory rate               41
temperature of extremities      4
peripheral pulse                4
mucous membranes                6
capillary refill time           3
pain                            6
peristalsis                     4
abdominal distension            4
nasogastric reflux              3
 packed cell volume            51
total protein                  82
surgical lesion?                2
1                              61
2                               6
3                               2
path                            2
dtype: int64

In [166]:
# feature_columns_con=['rectal temperature','pulse','respiratory rate',' packed cell volume','total protein']
feature_columns = ['rectal temperature','pulse','respiratory rate',' packed cell volume','total protein','1']
# feature_columns13=['rectal temperature', 'pulse', 'capillary refill time', 'pain', 'peristalsis', ' packed cell volume', 'total protein', '1', 'path']

for col in feature_columns:
    data[col] = pd.qcut(data[col], q=10, labels=False, duplicates='drop')

data.drop('Hospital ID', axis=1, inplace=True)

In [167]:
def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

In [181]:
def fitness_function(condition_attrs, data, decision_attr):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    equivalence_classes = calculate_equivalence_classes(data, condition_attrs)
    # print(len(equivalence_classes))
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB_sum=0
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(len(x))
            if(len(x)>total_records*0.006):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB = -class_prob * decision_prob * math.log(decision_prob, 2)
                entropy_DB_sum= entropy_DB_sum+entropy_DB

    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)

    print('--------------')
    print(count)
    print(total_rule)
    print(entropy_D-entropy_DB_sum)
    print(np.sum(rule_confidences) / count)
    if count == 0:
        return 999;
    if total_rule/total_records<0.15:
        return 999

    print('--------------')
    print(count)
    print(total_rule)
    print(entropy_D-entropy_DB_sum)
    print(np.sum(rule_confidences) / count)
    avg_confidence = np.sum(rule_confidences) / count +0.1*(entropy_D-entropy_DB_sum)

    return avg_confidence

In [185]:
# feature_columns=['cap-shape', 'cap-surface' ,'cap-color' ,'bruises' ,'odor',
#  'gill-attachment' ,'gill-spacing' ,'gill-size', 'gill-color', 'stalk-shape',
#  'stalk-root','stalk-surface-above-ring', 'stalk-surface-below-ring',
#  'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type',
#  'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population',
#  'habitat']
# for column in feature_columns:
#     print(column)
#     print(fitness_function(column, data, 'classes'))
# feature_columns=['spore-print-color', 'population', 'odor']
# feature_columns=['ring-type', 'stalk-color-below-ring', 'population', 'bruises', 'gill-size']
# feature_columns11=['f21', 'f10', 'f33', 'f32', 'f6', 'f35', 'f15', 'f1', 'f34', 'f7', 'f16', 'f23', 'f17', 'f4']
# feature_columns12=['f1', 'f2', 'f3', 'f5', 'f7', 'f9', 'f10', 'f11', 'f13', 'f15', 'f16', 'f17', 'f20', 'f21', 'f22', 'f23', 'f25', 'f26', 'f27', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35']
# feature_columns13=['f7', 'f10', 'f15', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f29', 'f32', 'f33', 'f35', 'f36']
# feature_columns14=['f1', 'f2', 'f6', 'f7', 'f10', 'f12', 'f13', 'f15', 'f16', 'f18', 'f21', 'f23', 'f26', 'f30', 'f32', 'f33', 'f35']
# feature_columns15=['f7', 'f10', 'f11', 'f12', 'f14', 'f16', 'f17', 'f20', 'f21', 'f23', 'f25', 'f26', 'f27', 'f28', 'f31', 'f32', 'f33', 'f34', 'f36']
# feature_columns16=['f1', 'f2', 'f3', 'f4', 'f6', 'f7', 'f10', 'f11', 'f15', 'f16', 'f17', 'f19', 'f21', 'f23', 'f25', 'f29', 'f33', 'f34', 'f35']
# feature_columns17=['f1', 'f2', 'f6', 'f10', 'f14', 'f15', 'f19', 'f21', 'f26', 'f28', 'f32', 'f33', 'f35', 'f36']
# feature_columns18=['f21', 'f10', 'f33', 'f32', 'f6', 'f35', 'f15', 'f1', 'f34', 'f7']
# feature_columns19=['f4', 'f6', 'f7', 'f10', 'f13', 'f15', 'f16', 'f17', 'f21', 'f22', 'f23', 'f27', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35']
# feature_columns20=['f1', 'f2', 'f4', 'f10', 'f11', 'f17', 'f21', 'f26', 'f28', 'f29', 'f30', 'f32', 'f33', 'f36']
# feature_columns13=['status_account', 'duration', 'present_emp', 'inst_plans']
# feature_columns14=['credit_history', 'residence_info', 'age', 'housing']
# feature_columns15=['duration', 'credit_history', 'age', 'inst_plans', 'dependents']
# feature_columns16=['status_account', 'other_debtors', 'age', 'inst_plans', 'num_credits', 'telephone']
feature_columns13=['rectal temperature', 'pulse', 'capillary refill time', 'pain', 'peristalsis', ' packed cell volume', 'total protein', '1', 'path']
feature_columns14=['rectal temperature', 'pulse', 'temperature of extremities', 'peripheral pulse', 'mucous membranes', 'capillary refill time', 'peristalsis', 'abdominal distension', 'nasogastric reflux', ' packed cell volume', 'total protein', '1', 'path']
feature_columns15=['Age', 'mucous membranes', 'nasogastric reflux', '1', 'path']
feature_columns16=['surgery', 'Age', 'temperature of extremities', 'peripheral pulse', 'mucous membranes', 'pain', 'abdominal distension', 'nasogastric reflux', 'total protein', '3', 'path']
feature_columns17=['pulse', 'respiratory rate', 'temperature of extremities', 'peristalsis', 'nasogastric reflux', ' packed cell volume', 'total protein', 'surgical lesion?', '1', '2', '3', 'path']
feature_columns18=['surgery', 'rectal temperature', 'peripheral pulse', 'pain', 'peristalsis', 'total protein', 'surgical lesion?', '1', '3']

print(fitness_function(feature_columns15 ,data, 'outcome'))

--------------
46
163
0.9561851222907156
0.23321256038647348
--------------
46
163
0.9561851222907156
0.23321256038647348
0.32883107261554506


In [171]:
def generate_population(size, feature_columns):
    # 初始化种群
    population = []
    for i in range(size):
        individual = random.sample(feature_columns, random.randint(3, len(feature_columns) - 1))
        population.append(individual)
    return population

def select_parents(population, fitness_values, elite_size):
    elite_indices = np.argsort(fitness_values)[:elite_size]
    elite_population = [population[i] for i in elite_indices]
    population_without_elite = [individual for i, individual in enumerate(population) if i not in elite_indices]
    # 将相应的适应度函数值也从 fitness_values 中删除
    fitness_values_without_elite = [fitness_values[i] for i in range(len(fitness_values)) if i not in elite_indices]
    probabilities = (1 / (np.array(fitness_values_without_elite) + [1e-10])) / sum(1 / (np.array(fitness_values_without_elite) + [1e-10]))
    parents = []
    for _ in range(len(population) - elite_size):
        parent1_index = np.random.choice(len(population_without_elite), p=probabilities)
        parent2_index = np.random.choice(len(population_without_elite), p=probabilities)
        parents.append(population_without_elite[parent1_index])
        parents.append(population_without_elite[parent2_index])
    parents.extend(elite_population)
    return parents


def crossover(parents, crossover_prob):
    offspring = []
    if(len(parents)%2!=0):
        parents.pop()
    for i in range(0, len(parents)-1, 2):
        if random.random() < crossover_prob:
            # 随机选择交叉点，注意不能选择第一个或最后一个位置
            crossover_point = random.randint(1, len(parents[i]) - 2)
            # 生成两个新的后代
            child1 = parents[i][:crossover_point] + parents[i+1][crossover_point:]
            child2 = parents[i+1][:crossover_point] + parents[i][crossover_point:]
            offspring.extend([child1, child2])
        else:
            # 如果没有进行交叉操作，则直接将两个父代加入后代中
            offspring.extend([parents[i], parents[i+1]])
    return offspring

def mutate(individual, mutation_prob, feature_columns):
    mutated_individual = individual.copy()

    for i in range(len(mutated_individual) - 1):
        if random.random() < mutation_prob:
            attr = mutated_individual[i]
            other_attrs = [col for col in feature_columns if col != attr]
            if other_attrs:
                mutated_individual[i] = random.choice(other_attrs)
    return mutated_individual

In [179]:
def run_genetic_algorithm(data, decision_attr, population_size=50, max_generations=100, crossover_prob=0.8, mutation_prob=0.2, elite_size=5):
    feature_columns = [col for col in data.columns if col not in ['classes','id']]
    # 初始化种群
    population = generate_population(population_size,feature_columns)
    # 计算每个个体的适应度函数值

    fitness_values = []
    for feature_columns in tqdm(population):
        fitness = fitness_function(feature_columns, data, decision_attr)
        fitness_values.append(fitness)
    # fitness_values = [fitness_function(feature_columns, data, decision_attr) for feature_columns in population]
    # 记录最优个体的适应度函数值和基因型
    best_fitness_value = min(fitness_values)
    best_individual = population[np.argmin(fitness_values)]
    # 进化过程
    for generation in tqdm(range(max_generations)):
        # 选择父母
        selected_parents = select_parents(population, fitness_values,elite_size)
        # 按一定概率进行交叉操作
        offspring = crossover(selected_parents, crossover_prob)
        # 按一定概率进行变异操作
        offspring = [mutate(individual, mutation_prob, feature_columns) for individual in offspring]
        # 计算每个个体的适应度函数值
        offspring_fitness_values = [fitness_function(individual, data, decision_attr) for individual in offspring]

        # 合并父代和子代
        combined_population = population + offspring
        combined_fitness_values = fitness_values + offspring_fitness_values

        # 按适应度函数值从小到大排序
        sorted_indices = np.argsort(combined_fitness_values)

        combined_population = [combined_population[i] for i in sorted_indices]
        combined_fitness_values = [combined_fitness_values[i] for i in sorted_indices]

        # 更新最优个体
        if combined_fitness_values[0] < best_fitness_value:
            best_fitness_value = combined_fitness_values[0]
            best_individual = combined_population[0]
        print("第轮最优个体")
        print(best_individual)
        print(best_fitness_value)
        # 选择新的种群
        population = combined_population[:population_size]
        fitness_values = [fitness_function(individual, data, decision_attr) for individual in population]
    return best_individual, best_fitness_value

In [173]:
import time
start_time = time.time()
print(run_genetic_algorithm(data, 'classes', population_size=50, max_generations=100, crossover_prob=0.8,
                            mutation_prob=0.01, elite_size=1))
end_time = time.time()
print(f"Execution time: {end_time - start_time} seconds")

  0%|          | 0/50 [00:00<?, ?it/s]


KeyError: 'classes'

In [ ]:
data = pd.read_csv(open('data/room.csv'))
data.replace('?', np.nan).fillna(method='pad')

In [ ]:
from sklearn.preprocessing import LabelEncoder
feature_columns=['cap-shape', 'cap-surface' ,'cap-color' ,'bruises' ,'odor',
 'gill-attachment' ,'gill-spacing' ,'gill-size', 'gill-color', 'stalk-shape',
 'stalk-root','stalk-surface-above-ring', 'stalk-surface-below-ring',
 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type',
 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population',
 'habitat']

labelencoder=LabelEncoder()
for col in feature_columns:
    data[col] = labelencoder.fit_transform(data[col])
data['classes'] = data['classes'].map({'e': 0, 'p': 1})

Y = data["classes"]
feature_columns=['spore-print-color', 'gill-spacing', 'gill-color', 'cap-surface', 'ring-type', 'stalk-shape', 'veil-type', 'gill-size', 'stalk-root']
# feature_columns=['spore-print-color', 'habitat', 'stalk-color-above-ring', 'gill-size', 'stalk-shape', 'cap-surface', 'ring-type', 'gill-attachment', 'ring-number'] #超参数为0.1
# feature_columns=['stalk-color-above-ring', 'stalk-surface-above-ring', 'habitat', 'bruises', 'odor', 'veil-color', 'stalk-root', 'ring-type']#超参数为0.2
#feature_columns=['habitat', 'bruises', 'odor', 'gill-size', 'cap-color', 'ring-type', 'gill-spacing', 'gill-attachment', 'spore-print-color']#超参数为0.3
#feature_columns=['bruises', 'cap-surface', 'odor', 'habitat', 'ring-type', 'cap-shape', 'stalk-surface-below-ring']#超参数为0.4
X = data[feature_columns]

In [ ]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X,Y,test_size=0.33,random_state=4)

In [ ]:
import math
from sklearn.svm import SVC,LinearSVC
from sklearn.preprocessing import StandardScaler

def calGamma(X):
    N = X.shape[0]
    L = np.array(range(N))
    np.random.shuffle(L)
    d = []
    for i in range(1,round(N*0.01)):
        for j in range(round(N*0.01) + 1, round(N*0.02)):
            xi = X[L[i],:]
            xj = X[L[j],:]
            d.append(sum((np.power(xi - xj,2))))
    d.sort()
    idx = len(d)
    dn = d[round((idx-1)*0.1)]
    df = d[round((idx-1)*0.9)]
    gamma = 1/(2*(df - dn)/(2*(math.log(df)-math.log(dn))))
    return gamma

def SVM(trainX,trainY,testX,testY):

    scaler = StandardScaler()
    scaler.fit(trainX)
    trainX3 = scaler.transform(trainX)
    gamma = calGamma(trainX3)
    testX3 = scaler.transform(testX)
    model = SVC(C=45, kernel='rbf', gamma = gamma)
    model.fit(trainX3, trainY)
    print("Score of the svm on the test data:", model.score(testX3, testY))
    return model.score(testX3, testY)

In [ ]:
SVM(trainX,trainY,testX,testY)
